# Worksheet 05

Name: Youxuan Ma

UID: U23330522

### Topics

- Cost Functions
- Kmeans

### Cost Function

Solving Data Science problems often starts by defining a metric with which to evaluate solutions were you able to find some. This metric is called a cost function. Data Science then backtracks and tries to find a process / algorithm to find solutions that can optimize for that cost function.

For example suppose you are asked to cluster three points A, B, C into two non-empty clusters. If someone gave you the solution `{A, B}, {C}`, how would you evaluate that this is a good solution?

Notice that because the clusters need to be non-empty and all points must be assigned to a cluster, it must be that two of the three points will be together in one cluster and the third will be alone in the other cluster.

In the above solution, if A and B are closer than A and C, and B and C, then this is a good solution. The smaller the distance between the two points in the same cluster (here A and B), the better the solution. So we can define our cost function to be that distance (between A and B here)!

The algorithm / process would involve clustering together the two closest points and put the third in its own cluster. This process optimizes for that cost function because no other pair of points could have a lower distance (although it could equal it).

### K means

a) (1-dimensional clustering) Walk through Lloyd's algorithm step by step on the following dataset:

`[0, .5, 1.5, 2, 6, 6.5, 7]` (note: each of these are 1-dimensional data points)

Given the initial centroids:

`[0, 2]`

### Dataset:
$[0, 0.5, 1.5, 2, 6, 6.5, 7]$

### Initial Centroids:
$[c_1 = 0, c_2 = 2]$

#### Step 1: Assignment Step

- For $c_1 = 0$, the closest points are $[0, 0.5]$.
- For $c_2 = 2$, the closest points are $[1.5, 2, 6, 6.5, 7]$.

So, the clusters after the assignment step are:
- Cluster 1: $[0, 0.5]$
- Cluster 2: $[1.5, 2, 6, 6.5, 7]$

#### Step 2: Update Step

- New centroid for Cluster 1, $c_1$: $ \frac{0 + 0.5}{2} = 0.25 $
- New centroid for Cluster 2, $c_2$: $ \frac{1.5 + 2 + 6 + 6.5 + 7}{5} = 4.6 $

So, the updated centroids are:
- $c_1 = 0.25$
- $c_2 = 4.6$

#### Step 3: Repeat Assignment

- For $c_1 = 0.25$, the closest points are now $[0, 0.5, 1.5, 2]$.
- For $c_2 = 4.6$, the closest points are now $[6, 6.5, 7]$.

Clusters after the reassignment:
- Cluster 1: $[0, 0.5, 1.5, 2]$
- Cluster 2: $[6, 6.5, 7]$

#### Step 4: Repeat Update

- New centroid for Cluster 1, $c_1$: $ \frac{0 + 0.5 + 1.5 + 2}{4} = 1 $
- New centroid for Cluster 2, $c_2$: $ \frac{6 + 6.5 + 7}{3} = 6.5 $

Updated centroids are:
- $c_1 = 1$
- $c_2 = 6.5$

#### Step 5: Check for Convergence
Repeat the assignment and update steps until the centroids do not change anymore, or the change is below a certain threshold, indicating that the algorithm has converged.

Since the centroids in Step 2 still differ from the ones in Step 4 by a lot, we proceed with repeating the assignment and update steps.

#### Step 6: Repeat Assignment

- For $c_1 = 1$, the closest points are now $[0, 0.5, 1.5, 2]$.
- For $c_2 = 6.5$, the closest points are now $[6, 6.5, 7]$.

Clusters after the reassignment:
- Cluster 1: $[0, 0.5, 1.5, 2]$
- Cluster 2: $[6, 6.5, 7]$

#### Step 7: Repeat Update

- New centroid for Cluster 1, $c_1$: $ \frac{0 + 0.5 + 1.5 + 2}{4} = 1 $
- New centroid for Cluster 2, $c_2$: $ \frac{6 + 6.5 + 7}{3} = 6.5 $

Updated centroids are:
- $c_1 = 1$
- $c_2 = 6.5$

#### Step 8: Check for Convergence
Now, we have that the newly updated centroids in Step 7 are the same as the ones in Step 4, indicating that the algorithm has converged.

Therefore, we get the final result of the clustering of the given dataset:

- Cluster 1: $[0, 0.5, 1.5, 2]$
- Cluster 2: $[6, 6.5, 7]$


b) Describe in plain english what the cost function for k means is.

The cost function for K-means evaluates how compact the clusters are by measuring how close the points within each cluster are to their respective centroids. The aim is to achieve as low a value as possible for this cost function, indicating that the algorithm has found a good grouping of the data points.

In other words, the cost function measures how well the K-means algorithm has performed in grouping similar data points into clusters. 

c) For the same number of clusters K, why could there be very different solutions to the K means algorithm on a given dataset?

This is often due to the Random Initialization:

K-means typically starts by randomly selecting K points from the dataset as the initial centroids. Since this selection is random, different runs of the algorithm can start with different initial centroids, thus leading to different clustering outcomes.

d) Does Lloyd's Algorithm always converge? Why / why not?

Yes, Lloyd's Algorithm always converges. The reasons are:
1. Finite Dataset:

- Given a finite dataset, there are a limited number of ways to partition the data points into K clusters. Each iteration of the algorithm refines the cluster assignments based on the current centroids, moving towards a more optimal partitioning. Since the number of possible partitions is finite, the algorithm cannot continue to change the cluster assignments indefinitely. 

2. Cost Function Decrease:

- The objective of K-means is to minimize the cost function. Each iteration of Lloyd's Algorithm (reassigning points to the nearest centroid and then updating centroids to the mean of the points in each cluster) is guaranteed to not increase the cost function. In most cases, it decreases the cost function unless the algorithm has already reached an optimal or locally optimal configuration where further iterations do not change the cluster assignments.

However, while Lloyd's Algorithm is guaranteed to converge, it may converge to a local minimum of the cost function rather than the global minimum. 


e) Follow along in class the implementation of Kmeans

In [4]:
import numpy as np
from PIL import Image as im
import matplotlib.pyplot as plt
import sklearn.datasets as datasets

centers = [[0, 0], [2, 2], [-3, 2], [2, -4]]
X, _ = datasets.make_blobs(n_samples=300, centers=centers, cluster_std=1, random_state=0)

class KMeans():

    def __init__(self, data, k):
        self.data = data
        self.k = k
        self.assignment = [-1 for _ in range(len(data))]
        self.snaps = []
    
    def snap(self, centers):
        TEMPFILE = "temp.png"

        fig, ax = plt.subplots()
        ax.scatter(X[:, 0], X[:, 1], c=self.assignment)
        ax.scatter(centers[:,0], centers[:, 1], c='r')
        fig.savefig(TEMPFILE)
        plt.close()
        self.snaps.append(im.fromarray(np.asarray(im.open(TEMPFILE))))

    def unassign_all(self):
        self.assignment = [-1 for _ in range(len(self.data))]
        
    def initialize(self):
        return self.data[np.random.choice(len(self.data), self.k, replace=False)]
    
    def assign(self, centers):
        for i in range(len(self.data)):
            temp_dist = float('inf')
            self.assignment[i] = -1
            for j in range(len(centers)):
                new_dist = self.dist(self.data[i], centers[j])
                if new_dist < temp_dist:
                    temp_dist = new_dist
                    self.assignment[i] = j
                    
    def calculate_new_centers(self):
        centers = []
        for j in range(self.k):
            cluster_j = self.data[
                np.array([i for i in range(len(self.data)) if self.assignment[i] == j])
            ]
            centers.append(np.mean(cluster_j, axis=0))
        return np.array(centers)
                
    def dist(self, x, y):
        return sum((x - y)**2)**.5
    
    
    def are_centers_diff(self, centers, new_centers):
        for i in range(len(centers)):
            if centers[i] not in new_centers:
                return True
        return False
    
    def lloyds(self):
        centers = self.initialize()
        self.assign(centers)
        self.snap(centers)
        new_centers = self.calculate_new_centers()
        while self.are_centers_diff(centers, new_centers):
            centers = new_centers
            self.snap(centers)
            self.unassign_all()
            self.assign(centers)
            new_centers = self.calculate_new_centers()
        return
    
    def is_unassigned(self, i):
        return self.assignment[i] == -1
            

kmeans = KMeans(X, 6)
kmeans.lloyds()
images = kmeans.snaps

images[0].save(
    'kmeans.gif',
    optimize=False,
    save_all=True,
    append_images=images[1:],
    loop=0,
    duration=500
)

